In [5]:
import numpy as np
import Backgammon as B
import agent as A
import flipped_agent as FA
import tensorflow as tf
import keras
import keras.layers as L

In [6]:
def get_cumulative_rewards(rewards, gamma = 0.99):
   
    rewards = np.array(rewards)
    R = np.zeros_like(rewards, dtype= "float32")
    r = 0.
    for i, reward in enumerate(reversed(rewards)):
        r += reward
        R[-(i + 1)] = r
        r *= gamma
        
    return R
    
    

In [7]:
def train_step(_states, _actions, _rewards):
    _cumulative_rewards = get_cumulative_rewards(_rewards)
    update.run({states: _states, 
                actions: _actions, 
                cumulative_rewards: _cumulative_rewards})

In [8]:
# Skilgreina inputs í model
states = tf.placeholder("float32", (None, 29), name = "states")
actions = tf.placeholder("int32", name = "action_ids")
cumulative_rewards = tf.placeholder("float32", name = "cumulative_rewards")

# Skilgreina model (arkitektúrinn skiptir litlu máli þangað til að þjálfunin gengur)
model = keras.models.Sequential()
model.add(L.Dense(32, activation = "relu"))
model.add(L.Dense(64, activation = "relu"))
model.add(L.Dense(1))



logits = model(states)
policy = tf.nn.softmax(logits)
log_policy = tf.nn.log_softmax(logits)

get_action_prob = lambda s: policy.eval({states: s})

indices = tf.stack([tf.range(tf.shape(log_policy)[0]),actions],axis=-1)
log_policy_for_actions = tf.gather_nd(log_policy,indices)


J = tf.reduce_mean(log_policy_for_actions * cumulative_rewards)
entropy = -tf.reduce_sum(tf.multiply(policy, log_policy), 1, name="entropy")
all_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

loss = -J - 0.1 * entropy

update = tf.train.AdamOptimizer().minimize(loss,var_list=all_weights)

In [ ]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())

In [10]:
# Spila leikinn
boards, moves, rewards = [], [], []
    
board = B.init_board()
player = 1

k = 1 # Halda utan um hvenær ég fæ villu
error = False


'''
Pælingin hér er að spila einn leik til enda og geyma öll boards og actions. 
Reikna svo eligibility trace og update'a modelið með öllu episodeinu.
'''

while True:
    dice = B.roll_dice()

    for i in range(1 + int(dice[0] == dice[1])):

        legal_moves, legal_boards = B.legal_moves(board, dice, 1)
        legal_boards = np.array([board for board in legal_boards])

        if len(legal_moves) == 0:
            break

        probs = get_action_prob(legal_boards)
        n_actions = probs.shape[0]
        probs = probs.reshape(n_actions)
        probs = probs / np.sum(probs)

        action = np.random.choice(np.arange(0, n_actions), 
                             p = probs)

        move = legal_moves[action]

        next_board = B.update_board(board = board, move = move.T, player = 1)

        #record session history to train later
        boards.append(next_board)
        moves.append(move)
        board = next_board

        if B.game_over(board):
            rewards.append(1)
            break
        else:
            rewards.append(0)
            
    board = FA.flip_board(board)
    k += 1
    if B.check_for_error(board):
        Error = True
        print("Error at game step ", k)
        break
        
if not Error:        
    train_step(boards, rewards)

Too many or too few pieces on board!
Error at game step  3
